In [1]:
from pandas import read_csv
from pandas import DataFrame
from datetime import datetime
from matplotlib import pyplot
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from pandas import concat
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from numpy import concatenate
from math import sqrt

C:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def parse(x):
    return datetime.strptime(x, '%Y %m %d %H')

dataset = read_csv('model_data.csv', 
                   parse_dates = [['year', 'month', 'day', 'hour']],
                  index_col = 0, date_parser = parse)


In [3]:
dataset.head(60)

,DALMP,LOAD,WIND,OUTAGES
year_month_day_hour,,,,
2014-01-01 01:00:00,28.17,35886.713705,7461.49,8492
2014-01-01 02:00:00,25.07,35587.349910,7281.48,8496
2014-01-01 03:00:00,24.93,35332.870128,7130.77,8496
2014-01-01 04:00:00,24.92,35211.064380,7336.28,8496
2014-01-01 05:00:00,25.07,35490.158245,7333.28,8496
2014-01-01 06:00:00,27.23,36203.492735,7407.28,8496
2014-01-01 07:00:00,36.17,37084.806113,7376.86,8496
2014-01-01 08:00:00,38.19,37397.188208,7262.05,8496
2014-01-01 09:00:00,40.14,37119.474985,7160.14,8496


In [9]:
data, names, targets = list(), list(), list()

target = 'DALMP'

for col in dataset.columns:
    temp = dataset[col]
    for i in range(50, 0, -1):
        data.append(temp.shift(i))
        names += [col + '(t-%d)' %(i)]
    if col is target:
        targets = temp
agg = concat(data, axis = 1)
agg.dropna(inplace=True)
agg.columns = names

In [10]:
agg.head(20)

,DALMP(t-50),DALMP(t-49),DALMP(t-48),DALMP(t-47),DALMP(t-46),DALMP(t-45),DALMP(t-44),DALMP(t-43),DALMP(t-42),DALMP(t-41),...,OUTAGES(t-10),OUTAGES(t-9),OUTAGES(t-8),OUTAGES(t-7),OUTAGES(t-6),OUTAGES(t-5),OUTAGES(t-4),OUTAGES(t-3),OUTAGES(t-2),OUTAGES(t-1)
year_month_day_hour,,,,,,,,,,,,,,,,,,,,,
2014-01-03 03:00:00,28.17,25.07,24.93,24.92,25.07,27.23,36.17,38.19,40.14,34.74,...,9749.0,9299.0,9318.0,9318.0,9318.0,8773.0,8773.0,8343.0,8423.0,8853.0
2014-01-03 04:00:00,25.07,24.93,24.92,25.07,27.23,36.17,38.19,40.14,34.74,30.22,...,9299.0,9318.0,9318.0,9318.0,8773.0,8773.0,8343.0,8423.0,8853.0,8853.0
2014-01-03 05:00:00,24.93,24.92,25.07,27.23,36.17,38.19,40.14,34.74,30.22,27.14,...,9318.0,9318.0,9318.0,8773.0,8773.0,8343.0,8423.0,8853.0,8853.0,8853.0
2014-01-03 06:00:00,24.92,25.07,27.23,36.17,38.19,40.14,34.74,30.22,27.14,25.06,...,9318.0,9318.0,8773.0,8773.0,8343.0,8423.0,8853.0,8853.0,8853.0,8853.0
2014-01-03 07:00:00,25.07,27.23,36.17,38.19,40.14,34.74,30.22,27.14,25.06,25.06,...,9318.0,8773.0,8773.0,8343.0,8423.0,8853.0,8853.0,8853.0,8853.0,8853.0
2014-01-03 08:00:00,27.23,36.17,38.19,40.14,34.74,30.22,27.14,25.06,25.06,24.92,...,8773.0,8773.0,8343.0,8423.0,8853.0,8853.0,8853.0,8853.0,8853.0,8853.0
2014-01-03 09:00:00,36.17,38.19,40.14,34.74,30.22,27.14,25.06,25.06,24.92,24.92,...,8773.0,8343.0,8423.0,8853.0,8853.0,8853.0,8853.0,8853.0,8853.0,8853.0
2014-01-03 10:00:00,38.19,40.14,34.74,30.22,27.14,25.06,25.06,24.92,24.92,25.00,...,8343.0,8423.0,8853.0,8853.0,8853.0,8853.0,8853.0,8853.0,8853.0,8853.0
2014-01-03 11:00:00,40.14,34.74,30.22,27.14,25.06,25.06,24.92,24.92,25.00,36.14,...,8423.0,8853.0,8853.0,8853.0,8853.0,8853.0,8853.0,8853.0,8853.0,8853.0


In [11]:
targets = dataset['DALMP'][agg.index]

In [12]:
agg['targets'] = targets

In [13]:
reframed = agg
reframed.head()

,DALMP(t-50),DALMP(t-49),DALMP(t-48),DALMP(t-47),DALMP(t-46),DALMP(t-45),DALMP(t-44),DALMP(t-43),DALMP(t-42),DALMP(t-41),...,OUTAGES(t-9),OUTAGES(t-8),OUTAGES(t-7),OUTAGES(t-6),OUTAGES(t-5),OUTAGES(t-4),OUTAGES(t-3),OUTAGES(t-2),OUTAGES(t-1),targets
year_month_day_hour,,,,,,,,,,,,,,,,,,,,,
2014-01-03 03:00:00,28.17,25.07,24.93,24.92,25.07,27.23,36.17,38.19,40.14,34.74,...,9299.0,9318.0,9318.0,9318.0,8773.0,8773.0,8343.0,8423.0,8853.0,28.26
2014-01-03 04:00:00,25.07,24.93,24.92,25.07,27.23,36.17,38.19,40.14,34.74,30.22,...,9318.0,9318.0,9318.0,8773.0,8773.0,8343.0,8423.0,8853.0,8853.0,28.42
2014-01-03 05:00:00,24.93,24.92,25.07,27.23,36.17,38.19,40.14,34.74,30.22,27.14,...,9318.0,9318.0,8773.0,8773.0,8343.0,8423.0,8853.0,8853.0,8853.0,30.78
2014-01-03 06:00:00,24.92,25.07,27.23,36.17,38.19,40.14,34.74,30.22,27.14,25.06,...,9318.0,8773.0,8773.0,8343.0,8423.0,8853.0,8853.0,8853.0,8853.0,40.00
2014-01-03 07:00:00,25.07,27.23,36.17,38.19,40.14,34.74,30.22,27.14,25.06,25.06,...,8773.0,8773.0,8343.0,8423.0,8853.0,8853.0,8853.0,8853.0,8853.0,76.75


In [11]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled = scaler.fit_transform(reframed.values)
scaled = DataFrame(scaled)
scaled.columns = reframed.columns
print(reframed.shape)



(40649, 201)


In [14]:
inv_reframed = scaler.inverse_transform(scaled)
inv_reframed

array([[  28.17,   25.07,   24.93, ..., 8423.  , 8853.  ,   28.26],
       [  25.07,   24.93,   24.92, ..., 8853.  , 8853.  ,   28.42],
       [  24.93,   24.92,   25.07, ..., 8853.  , 8853.  ,   30.78],
       ...,
       [  20.  ,   20.03,   23.  , ..., 4010.  , 3601.  ,   19.39],
       [  20.03,   23.  ,   24.55, ..., 3601.  , 3601.  ,   20.64],
       [  23.  ,   24.55,   26.78, ..., 3601.  , 3666.  ,   23.84]])

In [15]:
scaled_targets = scaled['targets']
scaled = scaled.drop(['targets'], axis = 1)
scaled.head()

,DALMP(t-50),DALMP(t-49),DALMP(t-48),DALMP(t-47),DALMP(t-46),DALMP(t-45),DALMP(t-44),DALMP(t-43),DALMP(t-42),DALMP(t-41),...,OUTAGES(t-10),OUTAGES(t-9),OUTAGES(t-8),OUTAGES(t-7),OUTAGES(t-6),OUTAGES(t-5),OUTAGES(t-4),OUTAGES(t-3),OUTAGES(t-2),OUTAGES(t-1)
0,0.012038,0.010649,0.010586,0.010582,0.010649,0.011617,0.015622,0.016527,0.017401,0.014981,...,0.293979,0.278390,0.279048,0.279048,0.279048,0.260168,0.260168,0.245271,0.248043,0.262939
1,0.010649,0.010586,0.010582,0.010649,0.011617,0.015622,0.016527,0.017401,0.014981,0.012956,...,0.278390,0.279048,0.279048,0.279048,0.260168,0.260168,0.245271,0.248043,0.262939,0.262939
2,0.010586,0.010582,0.010649,0.011617,0.015622,0.016527,0.017401,0.014981,0.012956,0.011576,...,0.279048,0.279048,0.279048,0.260168,0.260168,0.245271,0.248043,0.262939,0.262939,0.262939
3,0.010582,0.010649,0.011617,0.015622,0.016527,0.017401,0.014981,0.012956,0.011576,0.010645,...,0.279048,0.279048,0.260168,0.260168,0.245271,0.248043,0.262939,0.262939,0.262939,0.262939
4,0.010649,0.011617,0.015622,0.016527,0.017401,0.014981,0.012956,0.011576,0.010645,0.010645,...,0.279048,0.260168,0.260168,0.245271,0.248043,0.262939,0.262939,0.262939,0.262939,0.262939


In [16]:
values = scaled.values
values.shape

(40649, 200)

In [18]:
values = values.reshape((40649, 50, 4))
print(values.shape)
values

(40649, 50, 4)


array([[[0.01203794, 0.01064912, 0.0105864 , 0.01058192],
        [0.01064912, 0.01161681, 0.01562199, 0.01652696],
        [0.01740058, 0.01498134, 0.01295635, 0.01157649],
        ...,
        [0.27908266, 0.27908266, 0.29397908, 0.2783898 ],
        [0.27904801, 0.27904801, 0.27904801, 0.26016767],
        [0.26016767, 0.24527125, 0.24804268, 0.2629391 ]],

       [[0.01064912, 0.0105864 , 0.01058192, 0.01064912],
        [0.01161681, 0.01562199, 0.01652696, 0.01740058],
        [0.01498134, 0.01295635, 0.01157649, 0.01064464],
        ...,
        [0.27908266, 0.29397908, 0.2783898 , 0.27904801],
        [0.27904801, 0.27904801, 0.26016767, 0.26016767],
        [0.24527125, 0.24804268, 0.2629391 , 0.2629391 ]],

       [[0.0105864 , 0.01058192, 0.01064912, 0.01161681],
        [0.01562199, 0.01652696, 0.01740058, 0.01498134],
        [0.01295635, 0.01157649, 0.01064464, 0.01064464],
        ...,
        [0.29397908, 0.2783898 , 0.27904801, 0.27904801],
        [0.27904801, 0.260167

In [20]:
n_train_hours = 365*24*4
train_X = values[:n_train_hours, :, :]
test_X = values[n_train_hours:, :, ]
train_y = scaled_targets.values[:n_train_hours]
test_y = scaled_targets.values[n_train_hours:]

In [21]:
print("Training features:", train_X.shape)
print("Training targets:", train_y.shape)

print("Testing features:", test_X.shape)
print("Testing targets:", test_y.shape)


Training features: (35040, 50, 4)
Training targets: (35040,)
Testing features: (5609, 50, 4)
Testing targets: (5609,)


In [29]:
model = Sequential()
model.add(LSTM(100, input_shape = (50, 4), return_sequences = True))
model.add(LSTM(100, return_sequences = True))
model.add(LSTM(100))
model.add(Dense(1))
model.compile(loss = "mae", optimizer = 'adam', metrics = ['mse'])

history = model.fit(train_X, train_y, epochs = 10, batch_size = 256,
                   validation_data = (test_X, test_y),
                   shuffle = False)
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

Train on 35040 samples, validate on 5609 samples
Epoch 1/10
35040/35040 [==============================] - 154s 4ms/step - loss: 0.0085 - mean_squared_error: 3.3059e-04 - val_loss: 0.0145 - val_mean_squared_error: 0.0019
Epoch 2/10
35040/35040 [==============================] - 168s 5ms/step - loss: 0.0076 - mean_squared_error: 1.9416e-04 - val_loss: 0.0111 - val_mean_squared_error: 0.0019
Epoch 3/10
35040/35040 [==============================] - 168s 5ms/step - loss: 0.0047 - mean_squared_error: 1.6237e-04 - val_loss: 0.0076 - val_mean_squared_error: 0.0018
Epoch 4/10
35040/35040 [==============================] - 167s 5ms/step - loss: 0.0044 - mean_squared_error: 1.5610e-04 - val_loss: 0.0077 - val_mean_squared_error: 0.0018
Epoch 5/10
35040/35040 [==============================] - 168s 5ms/step - loss: 0.0040 - mean_squared_error: 1.5491e-04 - val_loss: 0.0077 - val_mean_squared_error: 0.0018
Epoch 6/10
27648/35040 [======================>.......] - ETA: 34s - loss: 0.0043 - mean_sq

KeyboardInterrupt: 

In [ ]:
yhat = model.predict(test_X)
print(yhat.shape)
print(yhat)

In [ ]:
test_y


In [ ]:
test_X2 = test_X.reshape((test_X.shape[0], 200))
n = test_X2.shape[1]

In [ ]:
inv_yhat = concatenate((test_X2[:, 0:],yhat), axis = 1)
print(inv_yhat.shape)
inv_yhat[:,n]

In [ ]:
test_y = test_y.reshape((len(test_y), 1))
y_actuals = concatenate((test_X2[:, 0:], test_y), axis = 1)

In [ ]:
y_actuals.shape
y_actuals[:, n]

In [ ]:
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yactuals = scaler.inverse_transform(y_actuals)

In [ ]:
print(inv_yhat.shape)
inv_yhat[:,n]

In [ ]:
print(inv_yactuals.shape)
inv_yactuals

In [ ]:
actuals = inv_yactuals[:,n]
predicted = inv_yhat[:,n]
results_df = DataFrame({'predicted':predicted,'actuals':actuals})
rmse = sqrt(mean_squared_error(results_df['actuals'], results_df['predicted']))
print('Test RMSE: %.3f' % rmse)

In [ ]:
pyplot.scatter(x = predicted, y = actuals, label='test')
pyplot.legend()
pyplot.show()

In [ ]:
model.evaluate()